In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from semantic_text_similarity.models import WebBertSimilarity
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import spatial
import parmap
import os
import swifter
from tqdm import tqdm

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


KeyboardInterrupt: 

In [ ]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [ ]:
bbc_id = 175
year = 2018

In [ ]:
path = '../data/bbc/{}/{}/transcripts'.format(bbc_id, year)
files = os.listdir(path)

data = []
for source in files:
    data.append(pd.read_csv(os.path.join(path, source)))
df = pd.concat(data)
df = df.drop(['Unnamed: 0', 'Has Transcript', 'Unavailable link', 'Unavailable reason'], axis=1)
df = df.reset_index(drop=True)
df

In [4]:
def filter_short_sentence(sentences):
    n_sent = len(sentences)
    filtered_sentences = []
    for i in range(n_sent):
        if len(sentences[i].split()) >= 11:
            pass
        else:
            filtered_sentences.append(sentences[i])
    return filtered_sentences

def get_short_sentences(transcript):
    cluster = []
    sentences = nltk.sent_tokenize(transcript)
    sentences = filter_short_sentence(sentences)
    
    return sentences

In [ ]:
transcripts = df['Transcript'].values

In [ ]:
short_sentences = []
for transcript in tqdm(transcripts):
    short_sentences.extend(get_short_sentences(transcript))

In [41]:
short_sentences[2]

'I don’t want this to be a show.'

In [42]:
len_short_sentences = [len(short_sentence) for short_sentence in short_sentences]


In [43]:
sum(len_short_sentences)

5993294

In [44]:
df['len_transcript'] = df['Transcript'].apply(lambda x: len(x.split()))

In [45]:
df['len_transcript'].sum()

7062293

In [46]:
sum(len_short_sentences) / df['len_transcript'].sum()

0.8486328732042129

In [47]:
with open('dropped_sentences_2015.txt', 'w') as f:
    f.writelines(short_sentences)

# Store dropped sentences as Others

In [8]:
years = [2015, 2016, 2017, 2018]
ids = [106, 107, 175, 279]

In [9]:
for year in years:
    for bbc_id in ids:
        print(year, bbc_id)
        path = '../data/bbc/{}/{}/transcripts'.format(bbc_id, year)
        files = os.listdir(path)

        data = []
        for source in files:
            data.append(pd.read_csv(os.path.join(path, source)))
        df = pd.concat(data)
        df = df.drop(['Unnamed: 0', 'Has Transcript', 'Unavailable link', 'Unavailable reason'], axis=1)
        df = df.reset_index(drop=True)
        dropped_rows = []

        for index, row in df.iterrows():
            sentences = ' '.join(get_short_sentences(row['Transcript']))
            dropped_row = [row['Source'], row['Date'], row['Program Name'], row['Time'], row['Duration'], sentences]
            dropped_rows.append(dropped_row)

        res_df = pd.DataFrame(dropped_rows, columns=['Source', 'Date', 'Program Name', 'Time', 'Duration', 'Transcript'])
        res_df.to_csv('dropped_sentences_{}_{}.csv'.format(bbc_id, year))

2015 106
2015 107
2015 175
2015 279
2016 106
2016 107
2016 175
2016 279
2017 106
2017 107
2017 175
2017 279
2018 106
2018 107
2018 175
2018 279
